In [ ]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# url = 'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey='
url = 'https://www.alphavantage.co/query?function=INFLATION&interval=monthly&apikey='
r = requests.get(url)
data = r.json()

print(data)

{'name': 'Inflation - US Consumer Prices', 'interval': 'annual', 'unit': 'percent', 'data': [{'date': '2024-01-01', 'value': '2.94952520485207'}, {'date': '2023-01-01', 'value': '4.11633838374488'}, {'date': '2022-01-01', 'value': '8.00279982052121'}, {'date': '2021-01-01', 'value': '4.69785886363742'}, {'date': '2020-01-01', 'value': '1.23358439630629'}, {'date': '2019-01-01', 'value': '1.81221007526021'}, {'date': '2018-01-01', 'value': '2.44258329692817'}, {'date': '2017-01-01', 'value': '2.13011000365961'}, {'date': '2016-01-01', 'value': '1.26158320570536'}, {'date': '2015-01-01', 'value': '0.118627135552451'}, {'date': '2014-01-01', 'value': '1.62222297740817'}, {'date': '2013-01-01', 'value': '1.46483265562717'}, {'date': '2012-01-01', 'value': '2.06933726526067'}, {'date': '2011-01-01', 'value': '3.156841568622'}, {'date': '2010-01-01', 'value': '1.6400434423899'}, {'date': '2009-01-01', 'value': '-0.355546266299747'}, {'date': '2008-01-01', 'value': '3.839100296651'}, {'date':

In [ ]:
https://api.worldbank.org/v2/country/vn/indicator/SP.POP.TOTL?date=2020&format=json

In [33]:
base_inflation_url = 'https://api.worldbank.org/v2/country/US/indicator/FP.CPI.TOTL.ZG?date=1950:2023&format=json&page=2'
COUNTRY_CODE = 'USA'
START_YEAR = 1950
END_YEAR = 2025
page = 1

In [34]:
full_url = f'https://api.worldbank.org/v2/country/{COUNTRY_CODE}/indicator/SP.POP.TOTL?date={START_YEAR}:{END_YEAR}&format=json'
import asyncio
import aiohttp
import pandas as pd
import json

async def fetch_data(session, url):
    async with session.get(url) as response:
        return await response.json()

async def main():
    async with aiohttp.ClientSession() as session:
        data = await fetch_data(session, full_url)
        # df = pd.json_normalize(data)
        # print(df)
        return data

In [41]:
import requests
response = requests.get(full_url)

In [44]:
type(response.content)

bytes

In [37]:
page = response.json()

In [40]:
page[1]

[{'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'},
  'country': {'id': 'US', 'value': 'United States'},
  'countryiso3code': 'USA',
  'date': '2024',
  'value': 340110988,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'},
  'country': {'id': 'US', 'value': 'United States'},
  'countryiso3code': 'USA',
  'date': '2023',
  'value': 336806231,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'},
  'country': {'id': 'US', 'value': 'United States'},
  'countryiso3code': 'USA',
  'date': '2022',
  'value': 334017321,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.POP.TOTL', 'value': 'Population, total'},
  'country': {'id': 'US', 'value': 'United States'},
  'countryiso3code': 'USA',
  'date': '2021',
  'value': 332099760,
  'unit': '',
  'obs_status': '',
  'decimal': 0},
 {'indicator': {'id': 'SP.POP.TOTL', 'va

In [ ]:
import requests

url = "https://api.worldbank.org/v2/country/USA/indicator/FP.CPI.TOTL.ZG?date=1950:2025&format=json"
r = requests.get(url)
data = r.json()[1]  # phần dữ liệu


2024 2.94952520485207 <class 'float'>
2023 4.11633838374488 <class 'float'>
2022 8.00279982052121 <class 'float'>
2021 4.69785886363742 <class 'float'>
2020 1.23358439630629 <class 'float'>


In [47]:
df = pd.DataFrame(data)

In [48]:
df.head()

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2024,2.949525,,,1
1,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2023,4.116338,,,1
2,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2022,8.002800,,,1
3,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2021,4.697859,,,1
4,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2020,1.233584,,,1


In [49]:
df = df.dropna()
df

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2024,2.949525,,,1
1,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2023,4.116338,,,1
2,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2022,8.002800,,,1
3,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2021,4.697859,,,1
4,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2020,1.233584,,,1
5,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2019,1.812210,,,1
6,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2018,2.442583,,,1
7,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2017,2.130110,,,1
8,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2016,1.261583,,,1
9,"{'id': 'FP.CPI.TOTL.ZG', 'value': 'Inflation, ...","{'id': 'US', 'value': 'United States'}",USA,2015,0.118627,,,1
